In [1]:
import yfinance as yf

In [2]:
sp500 = yf.Ticker('^GSPC')

In [3]:
#get stock info
sp500.info

{'exchange': 'SNP',
 'shortName': 'S&P 500',
 'exchangeTimezoneName': 'America/New_York',
 'exchangeTimezoneShortName': 'EDT',
 'isEsgPopulated': False,
 'gmtOffSetMilliseconds': '-14400000',
 'quoteType': 'INDEX',
 'symbol': '^GSPC',
 'messageBoardId': 'finmb_INDEXGSPC',
 'market': 'us_market',
 'previousClose': 3795.73,
 'regularMarketOpen': 3821.75,
 'twoHundredDayAverage': 4406.2266,
 'trailingAnnualDividendYield': None,
 'payoutRatio': None,
 'volume24Hr': None,
 'regularMarketDayHigh': 3913.65,
 'navPrice': None,
 'averageDailyVolume10Day': 4786310000,
 'totalAssets': None,
 'regularMarketPreviousClose': 3795.73,
 'fiftyDayAverage': 4065.845,
 'trailingAnnualDividendRate': None,
 'open': 3821.75,
 'toCurrency': None,
 'averageVolume10days': 4786310000,
 'expireDate': None,
 'yield': None,
 'algorithm': None,
 'dividendRate': None,
 'exDividendDate': None,
 'beta': None,
 'circulatingSupply': None,
 'startDate': None,
 'regularMarketDayLow': 3821.75,
 'priceHint': 2,
 'currency': 

In [4]:
# get historical market data
sp500_df = sp500.history(period='max')
sp500_df.head(10)


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1950-01-03,16.660000,16.660000,16.660000,16.660000,1260000,0,0
1950-01-04,16.850000,16.850000,16.850000,16.850000,1890000,0,0
1950-01-05,16.930000,16.930000,16.930000,16.930000,2550000,0,0
1950-01-06,16.980000,16.980000,16.980000,16.980000,2010000,0,0
1950-01-09,17.080000,17.080000,17.080000,17.080000,2520000,0,0
1950-01-10,17.030001,17.030001,17.030001,17.030001,2160000,0,0
1950-01-11,17.090000,17.090000,17.090000,17.090000,2630000,0,0
1950-01-12,16.760000,16.760000,16.760000,16.760000,2970000,0,0
1950-01-13,16.670000,16.670000,16.670000,16.670000,3330000,0,0


In [7]:
sp500_df = sp500_df.drop(columns=['Dividends', 'Stock Splits'], axis=1)
sp500_df

,Open,High,Low,Close,Volume
Date,,,,,
1950-01-03,16.660000,16.660000,16.660000,16.660000,1260000
1950-01-04,16.850000,16.850000,16.850000,16.850000,1890000
1950-01-05,16.930000,16.930000,16.930000,16.930000,2550000
1950-01-06,16.980000,16.980000,16.980000,16.980000,2010000
1950-01-09,17.080000,17.080000,17.080000,17.080000,2520000
...,...,...,...,...,...
2022-06-17,3665.899902,3707.709961,3636.870117,3674.840088,6954110000
2022-06-21,3715.310059,3779.649902,3715.310059,3764.790039,4124600000
2022-06-22,3733.889893,3801.790039,3717.689941,3759.889893,4342270000


In [9]:
sp500_df.to_csv('S&P_500.csv',float_format='%.2f')